## Transfer Learning in pretrained model using Sequential and Functional API 

In [1]:
#Importing Few Libraries
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten
from tensorflow.keras.models import Model,Sequential
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential


tf.__version__

'2.2.0'

### Including top layers

In [9]:
#Loading the pretrained model
IMG_SHAPE = (224, 224, 3) #providing the input shape
#if top is true then we have to provide the input shape as 224,244,3 and if its false then we can provide which ever input
# we want
VGG16_MODEL=applications.VGG16(input_shape=IMG_SHAPE,weights='imagenet')


In [21]:
#if we dont want to train the layers
for layers in VGG16_MODEL.layers:
    layers.trainable=False

In [12]:
#Checking for the layers
for layers in VGG16_MODEL.layers:
    print(layers.name,layers.trainable)

input_4 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False
flatten False
fc1 False
fc2 False
predictions False


### Using the Sequential model

In [4]:
model = Sequential()
model.add(VGG16_MODEL)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1000)              138357544 
Total params: 138,357,544
Trainable params: 138,357,544
Non-trainable params: 0
_________________________________________________________________


### Using Functional API

In [10]:
input_layer = Input(shape=(224,224,3))
VGG = VGG16_MODEL(input_layer)
model_functional = Model(inputs=input_layer,outputs=VGG)
model_functional.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 1000)              138357544 
Total params: 138,357,544
Trainable params: 138,357,544
Non-trainable params: 0
_________________________________________________________________


### Not including top layers

In [19]:
#Loading pretrained VGG16 and not including the top layers
VGG16_MODEL=applications.VGG16(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')

In [14]:
#if we want to train the last few models instead of 4 you can change it to which ever number of layers you want
for layers in VGG16_MODEL.layers[:-4]:  #training only last 4 layers from VGG16 model
    layers.trainable=False

In [15]:
#Checking for the layers
for layers in VGG16_MODEL.layers:
    print(layers.name,layers.trainable)

input_6 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


###  Adding few layer based on the reqired output using Sequential model

In [16]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(VGG16_MODEL)

top_model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Conv1'))
top_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid',data_format='channels_last',name='Pool1'))
top_model.add(Flatten(input_shape=VGG16_MODEL.output_shape[1:]))
top_model.add(Dense(units=64,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=32),name='FC1'))
top_model.add(Dense(units=32,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=33),name='FC2'))
top_model.add(Dense(units=16,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=3),name='Output'))

In [17]:
top_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
Conv1 (Conv2D)               (None, 5, 5, 256)         1179904   
_________________________________________________________________
Pool1 (MaxPooling2D)         (None, 2, 2, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
FC1 (Dense)                  (None, 64)                65600     
_________________________________________________________________
FC2 (Dense)                  (None, 32)                2080      
_________________________________________________________________
Output (Dense)               (None, 16)               

###  Using Functional API

In [23]:
input_layer = Input(shape=(224,224,3))#creating input layer
layer1 = VGG16_MODEL(input_layer) #creating the layer1 with input as input layer
Conv1 = Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Conv1')(layer1) #creating convolution layer
Pool1 = MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid',data_format='channels_last',name='Pool1')(Conv1) #creating pool layer
flatten = Flatten(data_format='channels_last',name='Flatten')(Pool1) #flattening layer
FC1 = Dense(units=64,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=32),name='FC1')(flatten)

#FC layer
FC2 = Dense(units=32,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=33),name='FC2')(FC1)

#output layer
Out = Dense(units=16,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=3),name='Output')(FC2)

#Creating a model
model = Model(inputs=input_layer,outputs=Out)